## VRP SHEDULING

In [102]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
import array
import numpy as np
import scipy as sp
from numpy import matrix

## Input data

In [223]:
## TEST DATA - SMALL USE CASE

# load dispatsh matrix - demand
    #LJ,MB,CE,KP
D = [11, 60, 12, 70]

# vehicle load capacity
C = [100, 100]

#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,MB,CE,KP
E = [
    [1, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 1],
    [0, 0, 1, 0, 1]
    ]

edges = len(E[1])
nodes = len (E)
#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]

#number of vehicles
V = np.size(C)

E

[[1, 1, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 1, 1], [0, 0, 1, 0, 1]]

In [148]:
# test matrix
E = [
    [11, 12, 13, 14, 15],
    [21, 22, 23, 24, 25],
    [31, 32, 33, 34, 35],
    [41, 42, 43, 44, 45]
    ]
#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]

In [474]:

# load dispatsh matrix - demand
    #LJ, CE, MB, KP, NM
D = [17, 6, 25, 37, 12]

# vehicle load capacity
C = [100, 100, 100]

#network graph, stopci = povezave 12, 23, 34, 45, 35, 13, vrstice = mesta
    #LJ,CE,MB,KP,NM
E = [
    [10, 0, 0, 0, 16, 11],
    [0, 21, 23, 0, 0, 26],
    [31, 32, 0, 0, 0, 0],
    [0, 0, 0, 44, 45, 0],
    [0, 0, 53, 54, 0, 0]]

#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]
#-E (negative values)
negE=np.negative(E)

#number of nodes
N = np.size(E,0)
#number of edges
M = np.size(E,1)
#number of vehicles
V = np.size(C)

#[0, 0, 1, 1, 0, 0]
E

[[10, 0, 0, 0, 16, 11],
 [0, 21, 23, 0, 0, 26],
 [31, 32, 0, 0, 0, 0],
 [0, 0, 0, 44, 45, 0],
 [0, 0, 53, 54, 0, 0]]

## Creating Constraint matrix A

In [224]:
# CONSTRAINT IV - UPDATED - there is od number of edges on cycles
sizeA11=edges*V+nodes*V
sizeA1=3*nodes*V+edges*V
A1 = np.zeros((sizeA1, sizeA11))
for k in range (0, V):
    for i in range (0, nodes):
        for j in range (0, edges):
            A1[i+k*nodes][j+k*edges]=E[i][j]
            A1[i+k*nodes+V*nodes][j+k*edges]=-E[i][j]
for ik in range (0, nodes*V):
    A1[ik][edges*V+ik]=-2
    A1[ik+V*nodes][edges*V+ik]=2
    A1[ik+V*nodes][edges*V+ik]=2

for ik in range (0, V*nodes):
    A1[ik+2*V*nodes,V*edges+ik]=-1
for jk in range (0, V*edges):
    A1[jk+3*V*nodes,jk]=-1

In [226]:
   #****UPDATED****
# CONSTRAINT II - A2 - the number of packets delivered on the node is equal to all total demand on the node
#number of rows: 2 * num. of nodes
#number of columns: num. nodes * num. of cycles
# variables Oki
# B vector = [D, -D]

sizeA2= 2*nodes
sizeA21= nodes*V
A2 = np.zeros((sizeA2, sizeA21))
for i in range (0, nodes):
    for k in range (0, V):
            A2[i,i+nodes*k]=1
            A2[i+nodes,i+nodes*k]=-1

In [227]:
   #****UPDATED****
#CONSTRAINT III A3 matrix: sum of load on each vehicles must not exceed vehicle load capacity
# B vector = [C, -C, [0,0, 0,...]]
# variables Oki
sizeA3=V+V*nodes
sizeA31=V*nodes

A3 = np.zeros((sizeA3, sizeA31))
for k in range (0, V):                                  # for each vehicle
    for i in range (0, nodes):                         #take the sum of load on enach node (sum on Ow variables)
        #print("j", j+i*V)       
        A3[k,i+k*nodes]=1
for ik in range (0, nodes*V):
    A3[V+ik,ik]=-1
   
# A23 - concatenating A2 and A3
A23=A2
for i in range (0, len(A3)):
    A23=np.vstack([A23, A3[i,:]])

In [228]:
#CONSTRAINT I - total number of all packets delivered is equal to summ of all D

size1= 2*nodes*V + edges*V   # number of columns in matrix A1+A2+A3, variables X, K, O
size2= V*edges*nodes         # number of additional columns in matrix A41, variables Aijk
# b vector = d (sum of all parcels). size = 1; D_all = np.sum(D)

# constraint 4.1. SUM for ijk -> (Eij * Aijk)
A41 = np.zeros((1, size1+size2))
l=0
for i in range (0, nodes):              # itteration by node in E 
    for j in range (0, edges):         # itteration by edges in E
        for k in range (0, V):
            A41[0,(size1+l)]=-(E[i][j])
            #print(size1+l)
            l=l+1

In [229]:
# Adding constraints 4.2.: Aijk - Cki*di <= 0 
size1= V * edges * nodes   
size2= 2*V*nodes + V*edges + V*nodes*edges
#b vector = [0,0,...0], size = nodes* edges * V
offsetA4=0
A42 = np.zeros((size1, size2))
for i in range (0, nodes):
    for j in range (0,edges): 
        for k in range (0, V):
            A42[i*edges*V+j*V+k, edges*k+j]=-D[i]
            A42[i*edges*V+j*V+k, 2*V*nodes + V*edges + i*edges*V+j*V+k]=1

In [230]:
#constraint 4.3.: Aijk - Oki <= 0
size1= V * edges * nodes   
size2= 2*V*nodes + V*edges + V*nodes*edges
A43 = np.zeros((size1, size2))
#b vector = [0,0,...0], size = 

offset= V*nodes+V*edges
offset1= 2*V*nodes+V*edges
for k in range (0, V):
    for j in range (0, edges):
        for i in range (0, nodes):
            A43[k*edges*nodes+j*nodes+i,offset+k*nodes+i]=-1
            A43[k*edges*nodes+j*nodes+i,offset1+k*edges*nodes+j*nodes+i]=1

In [232]:
# FINAL MATRIX  - A with all constraints
#concatenate A1 and A23 = A matrix
A1extend = np.c_[A1, np.zeros((len(A1), len(A23[0])))]
A23extend = np.c_[np.zeros((len(A23), len(A1[0]))), A23]

A123=A1extend.copy()
for i in range (0, len(A23)):
    A123=np.vstack([A123, A23extend[i,:]])

#concatenate A4 matrix
A4=A41.copy()
for i in range (0, len(A42)):
    A4=np.vstack([A4, A42[i,:]])
for i in range (0, len(A43)):
    A4=np.vstack([A4, A43[i,:]])

#Final concate A123 & A4
A123extend = np.c_[A123, np.zeros((len(A123), nodes*edges*V))]
A=A123extend.copy()
for i in range (0, len(A4)):
    A=np.vstack([A, A4[i,:]])

non_zero_rows = np.count_nonzero((A != 0).sum(1)); zero_rows = len(A) - non_zero_rows
print ("number of zero rows in A matirx =", zero_rows)
print ("number of variables in A =", len(A[1]))
print ("number of constraints in A =", len(A))


number of zero rows in A matirx = 0
number of variables in A = 66
number of constraints in A = 133


## Creating & declaring variables

In [233]:
# CREATE VARIABLES  X - vector with c11-cnn variables
# vsak vektor v matriki X1n = dolžine len(E). 
#Skupno število vseh vektorjev X1n = len (Et)
#X variables X[0] to X[len(E) * V -1] -  variables in objective function 
#K - X[len(E) * V] to X[len(E) * V + len(A1) -1]  slack variables
#Ow - cycles load dispatch variables - X[len(E) * V + len(Et)*V*2] to X[len(E) * V + len(Et)*V*3 -1 ] 

# C variables
X1 = []
for i in range (0, edges*V):
    var='x'+str(i)
    X1.append(var)

#K variables
K = []
for j in range (0, V*nodes):
    #creating vector with K variables
    var='k'+str(j)
    K.append(var)
    
#"Ow" variables
Ow = []
for j in range(0, V*nodes):   # numer of vehicles 
    var='Ow'+str(j)
    Ow.append(var)

#"Aijk" variables
Aijk = []
for i in range (0,nodes):
    for j in range(0, edges):   # numer of vehicles 
        for k in range (0, V):
            var='A'+str(i)+str(j)+str(k)
            Aijk.append(var)
    
#Final X vector with all variables
X=X1.copy()
for i in range (0,len(K)):
    X.append(K[i])
for j in range (0,len(Ow)):
    X.append(Ow[j])
for ijk in range (0,len(Aijk)):
    X.append(Aijk[ijk])
print("number of all variables =", len(X))
print("number of C variables =", len(X1))
print("number of K variables =", len(K))
print("number of O variables =", len(Ow))
print("number of Aijk variables =", len(Aijk))

number of all variables = 66
number of C variables = 10
number of K variables = 8
number of O variables = 8
number of Aijk variables = 40


In [234]:
#Declaring the solver
solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
x_min = -solver.infinity()                   # lower variables border 
x_max = solver.infinity()                  # Upper variables border                        
    
variables = [] 

for varN, xi_name in enumerate(X1):                          # declaring objective variables [C1.... Cn]
    variables.append(solver.IntVar(0.0, x_max, xi_name))
    
for varN, xi_name in enumerate(K):                          # declaring slack variables
    variables.append(solver.IntVar(0.0, x_max, xi_name))

for varN, xi_name in enumerate(Ow):                          # declaring load doispatch variables
    variables.append(solver.NumVar(0.0, x_max, xi_name))

for varN, xi_name in enumerate(Aijk):                          # declaring load doispatch variables
    variables.append(solver.NumVar(0.0, x_max, xi_name))
    
print('Number of variables created =', solver.NumVariables())
    #for variable in variables:
        #print('%s = %d' % (variable.name(), variable.solution_value())

Number of variables created = 66


## Declare constraints

In [235]:
#EQUATIONS COEFICIENTS 

#coeficients for constraint IV  -A1 matrix
b1=[]
for i in range (0, len(A1)):
    b1.append(0)
# coeficients for A2 matrix: b2+b3
b2=[]
b2 = D + [-val for val in D] 
# coeficient for A3 matrix
b31=C; b32=[]
for i in range (0, V*nodes):
    b32.append(0)

#coeficients for A23 constraint III
if len(b2+b31+b32) != len(A23):
    print("Error - wrong number of A23 constants!!!")

#coeficients for A4 matrix
b4=[-np.sum(D)]
for i in range (0, 2*V*nodes*edges):
    b4.append(0)

#D_all = np.sum(D)

# Final vector of constants - b    
b = b1 + b2 + b31 + b32 + b4
#b=np.array(b)
print("number of constants =", len(b))

number of constants = 133


In [236]:
#DECLARE CONSTRAINTS
#b = np.zeros((1, len(A)))

for rowN, row in enumerate(A):
    left_side = None
    for colN, coeff in enumerate(row):
        if coeff == 0:
            continue
        if left_side is None:
            left_side = coeff*variables[colN]
        else:
            left_side += coeff*variables[colN]
    if left_side is None and b[rowN] < 0:
    #if left_side is None and b[0,rowN] < 0:
        raise ValueError('Constraint ' + str(rowN) + ' cannot be satisfied!')
    if left_side is not None:
        #solver.Add(left_side <= t[0,rowN])
        solver.Add(left_side <= b[rowN])
        
print('Number of constraints added =', solver.NumConstraints())

Number of constraints added = 133


## Declare Objective Function

In [237]:
# declare objective function: x0+x1+x2+x3+x4+......

#C1 = [variables[0], variables[1], variables[2], variables[3], variables[4], variables[5]]
C = []
for i in range (0, len(X1)):
    C.append(variables[i])
    #print (variables[i])
objective = sum(C)
solver.Minimize(objective)

# Weights for prioritizing different variables
#wgt_X1 = 1.0
#X1_coeffs = [ wgt_X1 for _ in X1 ]
#for coeffN, coeff in enumerate(X1_coeffs):
#    if coeff != 0:
#        C += coeff*X1[coeffN] 

#solver.Maximize(x0 + x1 + x2 + x3 + x4 + x5 + x6 +x7 + x8 + x9)   

## Invoke the solver

In [238]:
result_status = solver.Solve()
    # The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
result_status

0

In [240]:
X

['x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'k0',
 'k1',
 'k2',
 'k3',
 'k4',
 'k5',
 'k6',
 'k7',
 'Ow0',
 'Ow1',
 'Ow2',
 'Ow3',
 'Ow4',
 'Ow5',
 'Ow6',
 'Ow7',
 'A000',
 'A001',
 'A010',
 'A011',
 'A020',
 'A021',
 'A030',
 'A031',
 'A040',
 'A041',
 'A100',
 'A101',
 'A110',
 'A111',
 'A120',
 'A121',
 'A130',
 'A131',
 'A140',
 'A141',
 'A200',
 'A201',
 'A210',
 'A211',
 'A220',
 'A221',
 'A230',
 'A231',
 'A240',
 'A241',
 'A300',
 'A301',
 'A310',
 'A311',
 'A320',
 'A321',
 'A330',
 'A331',
 'A340',
 'A341']

In [239]:
for i in range (0, len(X)):
    print('variable', i, variables[i].solution_value())

variable 0 0.0
variable 1 0.0
variable 2 0.0
variable 3 0.0
variable 4 0.0
variable 5 0.0
variable 6 1.0
variable 7 1.0
variable 8 0.0
variable 9 1.0
variable 10 0.0
variable 11 0.0
variable 12 0.0
variable 13 0.0
variable 14 1.0
variable 15 0.0
variable 16 1.0
variable 17 1.0
variable 18 11.0
variable 19 48.0
variable 20 0.0
variable 21 0.0
variable 22 0.0
variable 23 11.999999999999996
variable 24 12.0
variable 25 70.0
variable 26 0.0
variable 27 0.0
variable 28 0.0
variable 29 0.0
variable 30 0.0
variable 31 11.0
variable 32 0.0
variable 33 0.0
variable 34 0.0
variable 35 0.0
variable 36 0.0
variable 37 0.0
variable 38 0.0
variable 39 0.0
variable 40 0.0
variable 41 0.0
variable 42 0.0
variable 43 0.0
variable 44 0.0
variable 45 0.0
variable 46 0.0
variable 47 0.0
variable 48 0.0
variable 49 12.0
variable 50 0.0
variable 51 0.0
variable 52 0.0
variable 53 0.0
variable 54 0.0
variable 55 12.0
variable 56 0.0
variable 57 0.0
variable 58 0.0
variable 59 0.0
variable 60 0.0
variable 61 

In [120]:
assert solver.VerifySolution(1e-7, True)

print('Solution:')
print('Objective value =', solver.Objective().Value())
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

print('x0 =', variables[0].solution_value())
print('x1 =', variables[1].solution_value())

Solution:
Objective value = 0.0
Number of variables = 66
Number of constraints = 133

Advanced usage:
Problem solved in 54697.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes
x0 = 0.0
x1 = 0.0
